In [1]:
# Autogluon 설치 
# 'WARNING: The following packages were previously imported in this runtime'라는 문구시 런타임 다시 시작


!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

In [4]:
# google drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 경로설정
cd "/content/drive/MyDrive/DMC Project/Project2/유현준/"


SyntaxError: ignored

In [ ]:
## 압출풀기
# !unzip -qq "/content/drive/MyDrive/DMC Project/Project2/유현준/preprocessed_train_Auto.zip"
# !unzip -qq "/content/drive/MyDrive/DMC Project/Project2/유현준/prerocessed_test_Auto.zip"


replace preprocessed_train_Auto.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: unzip:  cannot find or open /content/drive/MyDrive/DMC Project/Project2/유현준/prerocessed_test_Auto.zip, /content/drive/MyDrive/DMC Project/Project2/유현준/prerocessed_test_Auto.zip.zip or /content/drive/MyDrive/DMC Project/Project2/유현준/prerocessed_test_Auto.zip.ZIP.


In [3]:
import pandas as pd
import numpy as np

In [5]:
# 파일 불러오기
train_df = pd.read_csv("/content/drive/MyDrive/DMC Project/Project2/유현준/preprocessed_train_Auto.csv")
test_df = pd.read_csv("/content/drive/MyDrive/DMC Project/Project2/유현준/prerocessed_test_Auto.csv")

In [6]:
train_df

,Unnamed: 0,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,totals.hits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.referralPath,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,_visitHour
0,433,Organic Search,4,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,New York,New York NY,New York,9,9.0,13.0,261.0,1.0,17.19,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,9
1,550,Referral,11,Chrome,Windows,False,desktop,Americas,Northern America,United States,California,Los Angeles CA,Irvine,15,12.0,38.0,285.0,1.0,13.00,(not set),(direct),(none),NaN,/,NaN,NaN,NaN,NaN,NaN,7
2,555,Organic Search,6,Safari,iOS,True,mobile,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,15,15.0,42.0,1044.0,1.0,65.30,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,22
3,597,Referral,17,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,18,16.0,77.0,514.0,1.0,25.23,(not set),(direct),(none),NaN,/,NaN,NaN,NaN,NaN,NaN,6
4,619,Referral,1,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,New York,New York NY,New York,21,20.0,62.0,487.0,1.0,16.31,(not set),(direct),(none),NaN,/,NaN,NaN,NaN,NaN,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,1707986,Referral,10,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,New York,New York NY,New York,61,48.0,NaN,2454.0,1.0,336.26,(not set),(direct),(none),NaN,/,NaN,NaN,NaN,NaN,NaN,6
18510,1707989,Referral,2,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,Texas,Austin TX,Austin,66,30.0,NaN,1375.0,1.0,606.95,(not set),(direct),(none),NaN,/,NaN,NaN,NaN,NaN,NaN,7
18511,1707998,Referral,1,Chrome,Macintosh,False,desktop,Asia,Eastern Asia,South Korea,Seoul,(not set),Seoul,122,90.0,NaN,5381.0,1.0,35.75,(not set),(direct),(none),NaN,/,NaN,NaN,NaN,NaN,NaN,9
18512,1707999,Direct,3,Firefox,Macintosh,False,desktop,Americas,Northern America,United States,New York,New York NY,New York,130,90.0,NaN,8811.0,1.0,72.39,(not set),(direct),(none),NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [8]:
# 불피요한 index 열 제거
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [9]:
# 종속변수 로그변환
train_df['totals.totalTransactionRevenue'] = np.log1p(train_df['totals.totalTransactionRevenue'])
test_df['totals.totalTransactionRevenue'] = np.log1p(test_df['totals.totalTransactionRevenue'])

In [10]:
# Autogluon 학습
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='totals.totalTransactionRevenue').fit(train_data=train_df)
predictions = predictor.predict(test_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20210711_035112/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210711_035112/"
AutoGluon Version:  0.2.0
Train Data Rows:    18514
Train Data Columns: 28
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (10.759668555013992, 0.7884573603642702, 4.15086, 1.06311)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
NumExpr defaulting to 2 threads.
	Available Memory:                    12673.31 MB
	Train Data (Original)  Memory Usage: 21.75 MB (0.2% of available memory)
	Inferring data type of each feature based 

In [11]:
label = 'totals.totalTransactionRevenue'
print("Summary of class variable: \n", train_df[label].describe())

Summary of class variable: 
 count    18514.000000
mean         4.150857
std          1.063111
min          0.788457
25%          3.399863
50%          3.985088
75%          4.700185
max         10.759669
Name: totals.totalTransactionRevenue, dtype: float64


In [12]:
y_test = test_df['totals.totalTransactionRevenue']  # values to predict
test_data_nolab = test_df.drop(columns=['totals.totalTransactionRevenue'])  # delete label column to prove we're not cheating
test_data_nolab.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,totals.hits,totals.pageviews,totals.timeOnSite,totals.sessionQualityDim,totals.transactions,trafficSource.referralPath,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,_visitHour
0,Direct,1,Chrome,Chrome OS,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,11,10.0,302.0,53,1.0,(not set),(not set),(direct),(none),(not set),(not set),NaN,NaN,NaN,NaN,22
1,Organic Search,1,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,15,11.0,169.0,10,1.0,(not set),(not set),google,organic,(not provided),(not set),NaN,NaN,NaN,NaN,10
2,Direct,1,Chrome,Windows,False,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,16,14.0,213.0,28,1.0,(not set),(not set),(direct),(none),(not set),(not set),NaN,NaN,NaN,NaN,0
3,Referral,3,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,Texas,Austin TX,Austin,17,14.0,2016.0,75,1.0,/,(not set),(direct),(none),NaN,(not set),NaN,NaN,NaN,NaN,0
4,Organic Search,1,Chrome,Android,True,mobile,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,18,16.0,1355.0,16,1.0,(not set),(not set),google,organic,(not provided),(not set),NaN,NaN,NaN,NaN,13


In [13]:
predictor.leaderboard(test_df, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,-0.910960,-0.923939,0.095384,0.054123,1.396603,0.095384,0.054123,1.396603,1,True,11
1,ExtraTreesMSE,-0.920519,-0.942765,0.670688,0.219292,9.852824,0.670688,0.219292,9.852824,1,True,7
2,RandomForestMSE,-0.924656,-0.954422,0.640067,0.216990,20.965189,0.640067,0.216990,20.965189,1,True,5
3,LightGBM,-0.925783,-0.921356,0.087337,0.049058,0.741251,0.087337,0.049058,0.741251,1,True,4
4,NeuralNetFastAI,-0.925889,-0.937261,0.146619,0.079420,38.969229,0.146619,0.079420,38.969229,1,True,8
5,XGBoost,-0.936644,-0.926452,0.124043,0.039195,6.589666,0.124043,0.039195,6.589666,1,True,9
6,LightGBMXT,-0.940839,-0.917750,0.162485,0.083053,3.009072,0.162485,0.083053,3.009072,1,True,3
7,WeightedEnsemble_L2,-1.007357,-0.912269,1.763602,0.787034,401.115574,0.039604,0.002232,0.428293,2,True,12
8,NeuralNetMXNet,-1.029414,-0.940191,1.246349,0.528797,287.707745,1.246349,0.528797,287.707745,1,True,10
9,KNeighborsUnif,-1.063628,-1.064475,0.106102,0.103653,0.022571,0.106102,0.103653,0.022571,1,True,1
